In [42]:
import subprocess

subprocess.run(["python3", "/Users/tolgasabanoglu/Desktop/github/spatiotemporal/scripts/parse_garmin.py"])

🔐 Logging into Garmin Connect...

📆 Fetching Garmin data from 2025-03-31 to 2025-06-29


📅 Date: 2025-03-31
📦 Fetching steps...
✅ Saved steps to ../data/raw/steps_2025-03-31.json
📦 Fetching sleep...
✅ Saved sleep to ../data/raw/sleep_2025-03-31.json
📦 Fetching stress...
✅ Saved stress to ../data/raw/stress_2025-03-31.json
📦 Fetching body_battery...
✅ Saved body_battery to ../data/raw/body_battery_2025-03-31.json
📦 Fetching heart_rate...
✅ Saved heart_rate to ../data/raw/heart_rate_2025-03-31.json

📅 Date: 2025-04-01
📦 Fetching steps...
✅ Saved steps to ../data/raw/steps_2025-04-01.json
📦 Fetching sleep...
✅ Saved sleep to ../data/raw/sleep_2025-04-01.json
📦 Fetching stress...
✅ Saved stress to ../data/raw/stress_2025-04-01.json
📦 Fetching body_battery...
✅ Saved body_battery to ../data/raw/body_battery_2025-04-01.json
📦 Fetching heart_rate...
✅ Saved heart_rate to ../data/raw/heart_rate_2025-04-01.json

📅 Date: 2025-04-02
📦 Fetching steps...
✅ Saved steps to ../data/raw/steps_2025-04-0

CompletedProcess(args=['python3', '/Users/tolgasabanoglu/Desktop/github/spatiotemporal/scripts/parse_garmin.py'], returncode=0)

In [43]:
import os
import json
from glob import glob
import pandas as pd

# Directory with your raw Garmin data
raw_dir = "/Users/tolgasabanoglu/Desktop/github/spatiotemporal/data/raw/"
json_files = sorted(glob(os.path.join(raw_dir, "*.json")))

# Known Garmin data prefixes
file_types = ["steps", "stress", "sleep", "body_battery", "heart_rate"]

audit_rows = []

for ftype in file_types:
    matching = [f for f in json_files if os.path.basename(f).startswith(ftype)]
    entry = {
        "type": ftype,
        "file_count": len(matching),
        "format": "❓",
        "has_date": False,
        "sample_date": "—",
        "top_keys": []
    }
    
    for file in matching:
        try:
            with open(file) as f:
                data = json.load(f)

                if isinstance(data, list) and data:
                    entry["format"] = "list"
                    entry["top_keys"] = list(data[0].keys())
                    date = (
                        data[0].get("calendarDate") or
                        data[0].get("startGMT") or
                        data[0].get("startTimestampGMT")
                    )
                    if date:
                        entry["has_date"] = True
                        entry["sample_date"] = str(date)[:10]
                    break

                elif isinstance(data, dict):
                    entry["format"] = "dict"
                    entry["top_keys"] = list(data.keys())
                    date = (
                        data.get("calendarDate") or
                        data.get("date") or
                        data.get("startTimestampGMT")
                    )
                    if date:
                        entry["has_date"] = True
                        entry["sample_date"] = str(date)[:10]
                    break
        except Exception as e:
            entry["top_keys"] = [f"⚠️ Error: {e}"]

    audit_rows.append(entry)

# Show audit results as table
df_audit = pd.DataFrame(audit_rows)
df_audit = df_audit[["type", "file_count", "format", "has_date", "sample_date", "top_keys"]]

# Print as plain table
print("\n📊 Garmin JSON File Audit:\n")
print(df_audit.to_string(index=False))



📊 Garmin JSON File Audit:

        type  file_count format  has_date sample_date                                                                                                                                                                                                                                                                                                                                          top_keys
       steps          62   list      True  2025-03-30                                                                                                                                                                                                                                                                    [startGMT, endGMT, steps, pushes, primaryActivityLevel, activityLevelConstant]
      stress          62   dict      True  2025-03-31                                           [userProfilePK, calendarDate, startTimestampGMT, endTimestampGMT, startTimestampLocal, endTi

In [44]:
import os
import json
import pandas as pd
from glob import glob
from collections import defaultdict

# ---- Setup ----
raw_dir = "/Users/tolgasabanoglu/Desktop/github/spatiotemporal/data/raw/"
json_files = sorted(glob(os.path.join(raw_dir, "*.json")))
summary = defaultdict(dict)

# ---- Parse Files ----
for file in json_files:
    fname = os.path.basename(file)

    try:
        with open(file) as f:
            data = json.load(f)
    except Exception as e:
        print(f"⚠️ Could not load {fname}: {e}")
        continue

    # ---- STEPS ----
    if fname.startswith("steps") and isinstance(data, list) and data:
        date = data[0].get("startGMT", "")[:10]
        steps = sum(item.get("steps", 0) for item in data if isinstance(item, dict))
        summary[date]["steps"] = steps

    # ---- STRESS ----
    elif fname.startswith("stress") and isinstance(data, dict):
        date = data.get("calendarDate")
        if date:
            summary[date]["avg_stress"] = data.get("avgStressLevel")
            summary[date]["max_stress"] = data.get("maxStressLevel")

    # ---- BODY BATTERY ----
    elif fname.startswith("body_battery"):
        if isinstance(data, list):
            for entry in data:
                if isinstance(entry, dict):
                    date = entry.get("date")
                    if date:
                        summary[date]["body_charged"] = entry.get("charged")
                        summary[date]["body_drained"] = entry.get("drained")
        elif isinstance(data, dict):
            date = data.get("date")
            if date:
                summary[date]["body_charged"] = data.get("charged")
                summary[date]["body_drained"] = data.get("drained")

    # ---- HEART RATE ----
    elif fname.startswith("heart_rate") and isinstance(data, dict):
        date = data.get("calendarDate")
        if date:
            summary[date]["resting_hr"] = data.get("restingHeartRate")

    # ---- SLEEP ----
    elif fname.startswith("sleep") and isinstance(data, dict):
        record = {}
        date = None

        daily = data.get("dailySleepDTO")
        if isinstance(daily, dict):
            date = daily.get("calendarDate") or daily.get("sleepEndTimestampGMT", "")[:10]
            record.update({
                "sleep_score": daily.get("overallSleepScore"),
                "total_sleep_s": daily.get("sleepTimeSeconds"),
                "deep_sleep_s": daily.get("deepSleepSeconds"),
                "light_sleep_s": daily.get("lightSleepSeconds"),
                "rem_sleep_s": daily.get("remSleepSeconds"),
                "awake_s": daily.get("awakeSleepSeconds"),
                "restless_moments": daily.get("restlessMomentsCount")
            })

        rem = data.get("remSleepData")
        if isinstance(rem, dict):
            record.update({
                "rem_count": rem.get("remSleepCount"),
                "avg_rem_duration_s": rem.get("avgRemSleepSeconds")
            })

        if date:
            summary[date].update(record)

# ---- Create DataFrame ----
df = pd.DataFrame.from_dict(summary, orient="index").reset_index()
df = df.rename(columns={"index": "date"})
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.sort_values("date")

# ---- Convert seconds → hours ----
sleep_cols = ["total_sleep_s", "deep_sleep_s", "light_sleep_s", "rem_sleep_s", "awake_s", "avg_rem_duration_s"]
for col in sleep_cols:
    if col in df.columns:
        df[col.replace("_s", "_h")] = df[col] / 3600

# ---- Preview ----
print(df.head(10))


         date  body_charged  body_drained  resting_hr  sleep_score  \
62 2025-03-30           NaN           NaN         NaN          NaN   
0  2025-03-31          75.0          62.0        56.0          NaN   
1  2025-04-01          31.0          63.0        63.0          NaN   
2  2025-04-02          27.0          30.0        65.0          NaN   
3  2025-04-03          27.0          27.0        64.0          NaN   
4  2025-04-04          64.0          61.0        62.0          NaN   
5  2025-04-05          60.0          38.0        63.0          NaN   
6  2025-04-06          77.0          52.0        57.0          NaN   
7  2025-04-07          39.0          56.0        60.0          NaN   
8  2025-04-08          55.0          74.0        56.0          NaN   

    total_sleep_s  deep_sleep_s  light_sleep_s  rem_sleep_s  awake_s  \
62            NaN           NaN            NaN          NaN      NaN   
0         29015.0        3840.0        19860.0       5340.0    180.0   
1         295

In [56]:


# Assume df is already your loaded Garmin summary DataFrame
# If not, load it first (e.g., from a CSV or JSON)

# ---- 1. Remove unwanted columns ----
columns_to_drop = ["sleep_score", "restless_moments"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors="ignore")

# ---- 2. Ensure date is datetime and sorted ----
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.sort_values("date")

# ---- 3. Forward-fill then backward-fill missing values ----
df = df.fillna(method="ffill").fillna(method="bfill")

# ---- 4. Preview cleaned data ----
print(df.head())

# Optional: save to CSV
# df_filled.to_csv("garmin_cleaned.csv", index=False)


         date  body_charged  body_drained  resting_hr  total_sleep_s  \
62 2025-03-30          75.0          62.0        56.0        29015.0   
0  2025-03-31          75.0          62.0        56.0        29015.0   
1  2025-04-01          31.0          63.0        63.0        29520.0   
2  2025-04-02          27.0          30.0        65.0        27327.0   
3  2025-04-03          27.0          27.0        64.0        34260.0   

    deep_sleep_s  light_sleep_s  rem_sleep_s  awake_s   steps  avg_stress  \
62        3840.0        19860.0       5340.0    180.0  2258.0        30.0   
0         3840.0        19860.0       5340.0    180.0  8971.0        30.0   
1         5460.0        21000.0       3060.0   8280.0  3074.0        54.0   
2         4620.0        22020.0        720.0   2460.0  4237.0        51.0   
3         4200.0        22200.0       7860.0     60.0  3887.0        53.0   

    max_stress  total_hleep_h  deep_hleep_h  light_hleep_h  rem_hleep_h  \
62        94.0       8.059722

/var/folders/jg/x92gk5yd0t560nh7hmhyvm040000gn/T/ipykernel_1475/3967524136.py:13: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [57]:
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Ensure steps column is numeric
df["steps"] = pd.to_numeric(df["steps"], errors="coerce")

# Select valid numeric columns to compare with steps
columns_to_plot = [
    col for col in df.columns
    if col not in ["date", "steps"] and pd.api.types.is_numeric_dtype(df[col])
]

# Filter out columns with insufficient non-null pairings
valid_pairs = []
for col in columns_to_plot:
    temp_df = df[["steps", col]].dropna()
    if len(temp_df) >= 2:
        valid_pairs.append(col)

# Prepare plot
fig = sp.make_subplots(
    rows=len(valid_pairs), cols=1, shared_xaxes=False,
    subplot_titles=[f"Steps vs. {col.replace('_', ' ').title()}" for col in valid_pairs]
)

# Plot with regression
for i, col in enumerate(valid_pairs, start=1):
    temp_df = df[["steps", col]].dropna()
    X = temp_df["steps"].values.reshape(-1, 1)
    y = temp_df[col].values

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)

    fig.add_trace(
        go.Scatter(x=temp_df["steps"], y=temp_df[col], mode='markers', name=col),
        row=i, col=1
    )

    fig.add_trace(
        go.Scatter(x=temp_df["steps"], y=y_pred, mode='lines',
                   line=dict(dash='dash'), name=f"{col} Fit"),
        row=i, col=1
    )

    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.95, y=1 - (i - 1) / len(valid_pairs),
        text=f"R² = {r2:.2f}", showarrow=False, font=dict(size=12)
    )

fig.update_layout(
    height=350 * len(valid_pairs),
    width=1000,
    title_text="Steps vs Other Metrics with Regression Lines and R²",
    showlegend=False
)

fig.show()
